<a href="https://colab.research.google.com/github/camoragaq/C-uoh/blob/main/Clase_Pipes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pipes en UNIX

Usando pipe para escribir y leer en el archivo de descriptores

In [5]:
%%writefile pipe.c

#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>

#define MSGSIZE 16
char* msg1 = "hola mundo #1";
char* msg2 = "hola mundo #2";
char* msg3 = "hola mundo #3";

int main()
{
	char inbuf[MSGSIZE];
	int p[2], i;

	if (pipe(p) < 0)
		exit(1);


	write(p[1], msg1, MSGSIZE);
	write(p[1], msg2, MSGSIZE);
	write(p[1], msg3, MSGSIZE);

	for (i = 0; i < 3; i++) {

		read(p[0], inbuf, MSGSIZE);
		printf("%s\n", inbuf);
	}
	return 0;
}


Overwriting pipe.c


In [6]:
!gcc pipe.c -o pipe

In [7]:
!./pipe

hola mundo #1
hola mundo #2
hola mundo #3


In [11]:
%%writefile pipe1.c

#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <sys/wait.h>
#include <string.h>

int main(){
  pid_t pid;

  int fds[2];
  pipe(fds);

  pid = fork();//clonamos el proceso

  if(pid==-1){
    printf("Error en fork\n");
    exit(1);
  }
  else if(pid == 0){//estamos en el hijo
    close(fds[0]);//hijo envia datos
    char *string= "Hola alumnos!\n";
    write(fds[1],string, strlen(string)+1);

    exit(0);
  }
  else{
    close(fds[1]);//estamos en el padre y leerá los datos
    char buffer[80];
    int n = read(fds[0],buffer,80);

    waitpid(pid, NULL, 0);

    printf("Mensaje del hijo al padre: %s\n", buffer);
  }
}

Writing pipe1.c


In [12]:
!gcc pipe1.c -o pipe1

In [13]:
!./pipe1

Mensaje del hijo al padre: Hola alumnos!



Exec()
para ejecutar un comando por código

In [23]:
%%writefile pipe-fork.c

#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <sys/wait.h>
#include <sys/types.h>

int main(){
  pid_t pid;

  printf("-------------\n");
  fflush(stdout);

  pid = fork();
  if(pid <0){
    fprintf(stderr, "fork Error\n");
    exit(1);
  }
  if(pid == 0){
    execl("/bin/ls", "ls", "-thlr",NULL);
    fprintf(stderr, "exec...\n");
    exit(1);
  }
  else{
    int status;
    waitpid(pid,&status,0);
    printf("--------------\n");
    return 0;
  }
}

Overwriting pipe-fork.c


In [24]:
!gcc pipe-fork.c -o pipefork

In [ ]:
!./pipefork

In [9]:
!gcc pipe-fork.c -o pipefork

In [10]:
!./pipefork

^C
